In [62]:
import pandas as pd

In [15]:
filepath = "data/2021_NCVR_Panse_001/dataset_ncvr_dirty.csv"
col_names = "sourceID,globalID,localID,FIRSTNAME,MIDDLENAME,LASTNAME,YEAROFBIRTH,PLACEOFBIRTH,COUNTRY,CITY,PLZ,STREET,GENDER,ETHNIC,RACE".split(",")
df = pd.read_csv(filepath, names=col_names)
df.shape[0]

200000

In [2]:
import mlflow

In [4]:
exp_id = mlflow.create_experiment(tags={"id": "1", "l":1000, "k":15, "t":0.7})
exp_id

TypeError: create_experiment() missing 1 required positional argument: 'name'

In [ ]:
exp_id = mlflow.create_experiment(tags={"id": "1", "l":1000, "k":15, "t":0.7})
exp_id